In [1]:
#importing library
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet,RidgeCV,LassoCV,ElasticNetCV

In [2]:
#loading the dataset
df=pd.read_csv('../input/insurance/insurance.csv')
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [3]:
#looking for the null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [4]:
#Checking for the missing value
df.isna().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [11]:
print("Total non-numeric columns",len(df.select_dtypes('object').columns))

Total non-numeric columns 3


In [13]:
{column:list(df[column].unique()) for column in df.select_dtypes('object').columns}

{'sex': ['female', 'male'],
 'smoker': ['yes', 'no'],
 'region': ['southwest', 'southeast', 'northwest', 'northeast']}

In [14]:
def binary_encode(df,column,positive_value):
    df=df.copy()
    df[column]=df[column].apply(lambda x:1 if x==positive_value else 0)
    return df

In [21]:
def onehot_encode(df,column,prefix):
    df=df.copy()
    dummies=pd.get_dummies(df[column],prefix=prefix)
    df=pd.concat([df,dummies],axis=1)
    df=df.drop(column,axis=1)
    return df

In [24]:
def preprocess_inputs(df,scaler,train_size=0.7):
    df=binary_encode(df,'sex','male')
    df=binary_encode(df,'smoker','yes')
    df=onehot_encode(df,'children','ch')
    df=onehot_encode(df,'region','re')
    y=df['charges'].copy()
    x=df.drop('charges',axis=1).copy()
    x=pd.DataFrame(scaler.fit_transform(x),columns=x.columns)
    x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=train_size,random_state=1)
    return x_train,x_test,y_train,y_test

In [25]:
x_train,x_test,y_train,y_test=preprocess_inputs(df,StandardScaler(),train_size=0.7)

In [26]:
x_train.shape

(936, 14)

In [27]:
x_test.shape

(402, 14)

In [28]:
models={
    "Linea":LinearRegression(),
    "Ridge":Ridge(),
    "Lasso":Lasso(),
    "Elast":ElasticNet(),
    "Ridge":RidgeCV(),
    "Lasso":LassoCV(),
    "Elast":ElasticNetCV()
}

In [30]:
for model in models.values():
    model.fit(x_train,y_train)

In [33]:
print("Model R^2 Score is................")
for name,model in models.items():
    print(name,model.score(x_test,y_test))

Model R^2 Score is................
Linea 0.7380138769361969
Ridge 0.7380257908998857
Lasso 0.7379635391519065
Elast 0.1371255763787983
